In [1]:
 # if plotting does not work, change it into %matplotlib inline from %matplotlib widget
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from utilities import display_array, normalise_clip
import flammkuchen as fl

**What is the package flammkuchen?**

# Load and display the data

In [3]:
fps = 3
# Change this to the folder where you have the data
data_path = r"C:\Users\rportugues\tum_imaging_analysis_course\2p_nuclear\imaging.h5"
ar_imaging = fl.load(data_path)
display_array(ar_imaging);

interactive(children=(IntSlider(value=279, description='t', max=559), IntSlider(value=1, description='z', max=…

**Create an image of the anatomy by averaging in time and call it *anatomy***

In [5]:
# normalize the data
anatomy_normalized = normalise_clip(anatomy, wins_min=5, wins_max=99.5)

In [6]:
display_array(anatomy_normalized)

interactive(children=(IntSlider(value=1, description='i', max=2), Output()), _dom_classes=('widget-interact',)…

<function utilities.display_array.<locals>.browse(i: (0, 2))>

**Design a kernel similar to a cell for cross correlation. For indicators localised to the nucleus, like the one in this dataset, a gaussian might be a good fit.**

In [25]:
# creating a kernel simi for cross correlation 
# we are using a nuclear localized calcium indicator, so a good shape for that is a gaussian. 
import scipy 

shape = (12, 12)
m,n = [(ss-1.)/2. for ss in shape]
y,x = np.ogrid[-m:m+1,-n:n+1]
sigma = 1.5
h = np.exp( -(x**2 + y**2) / (2.*sigma**2) )
h /= np.sum(h)

#cell_pattern_1 = anat_clean[0]
#plt.imshow(cell_pattern_1)
plt.figure()
plt.imshow(h)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Convolve the kernel with the image. The correlate2d function from scipy.signal works for that**

In [28]:
# cross-correlation with the cell pattern in all planes
from scipy.signal import correlate2d

anatomy_corr = np.zeros_like(anatomy_normalized)
for i in range(3):
    #print(np.shape(h))
    #print(np.shape(anat_clean[i]))
    anatomy_corr[i] = correlate2d(anatomy_normalized[i], h, mode='same')

display_array(np.concatenate([anatomy_corr, anatomy_normalized], 2))

interactive(children=(IntSlider(value=1, description='i', max=2), Output()), _dom_classes=('widget-interact',)…

<function utilities.display_array.<locals>.browse(i: (0, 2))>

**Use local maxima to detect potential ROIs. Save them in a structure** coords[0:2][0:N-1,0:1] **where the first index labels the plane, the second index labels the number of the ROI within the plane, and the third index labels the X or Y coordinate. Use a 3D maximum and a relative threshold of 0.5. Plot the location of the maxima you found superimposed on the anatomy (for one plane) save the figure and submit it.** 

In [ ]:
from skimage.feature import peak_local_max

In [10]:
fig.savefig("MY_template_match.png")

**You should get something like:**

![](template_match.png)

## Extract traces around the points

**You can loop around the maxima coordinates and take a small rectangular window around it. For better results use a gaussian kernel. Save the traces of each cell in a matrix called** traces **that has in each row the fluorescence time series for one cell. The matrix will therefore be N x T, where N is the number of cells and T the number of timepoints.**

**You should get an array of traces [n_cells, n_timepoints]**

In [12]:
time = np.arange(traces.shape[1])/fps

**Instead of plotting raw fluorescence or delta fluorescence / fluorescence [df/f], we will plot the normalized fluorescence or "Zscored" fluorescence. You can easily compute this using:**  

In [29]:
from scipy.stats import zscore

**Save the image and submit it.**

In [14]:
fig.savefig("MY_traces_anat_1.png")

**The result should look a bit like this:**

![](traces_anat_1.png)

**Use ipywidgets interact to browse through the individual traces**

In [15]:
from ipywidgets import interact

In [16]:
fig, ax = plt.subplots()
@interact
def browse_trace(i_trace:(0, traces.shape[0]-1)):
    ax.clear()
    ax.plot(time, traces[i_trace])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=32, description='i_trace', max=64), Output()), _dom_classes=('widget-int…

## Correcting motion artifacts within a plane using two-dimensional cross-correlation
**Now we will load a tiff stack. This is a file that contains a set of images. It is therefore a three dimensional array (Z,X,Y) or (T,X,Y) depending on whether the different images are from different planes or the same plane at different times.**

In [18]:
from PIL import Image

img = Image.open("plane45.tif")
images = []
for i in range(img.n_frames):
    img.seek(i)
    images.append(np.array(img))
np.shape(images)

(1750, 400, 414)

**Create an anatomy image and display it**

**Now we will use the function** phase_cross_correlation **from ski-image. We want to perform sub-pixel corrections, say to 0.2 of a pixel, so make sure you use the** upsample_factor **argument. Save the shifts in the X and Y directions in two arrays and compute also a total motion array (use pythagoras).**

In [26]:
from skimage.registration import phase_cross_correlation
import math

**Plot the total motion of each frame. Do you think the motion arises from continuous drifts or acute motor events?**

### Second order correction of motion artifact
**Correct the individual frames by performing an individual sub-pixel alignment frame by frame. Use this new corrected stack to generate a new anatomy. Align THE ORIGINAL FRAMES to this new anatomy. Save this new "motion distance array" as a csv file (which should have 1750 entries) and submit it and the plot. Submit also an image with the three anatomies: the one before any motion correction, the one after one round of motion correction and the one after two rounds of motion correction.**

## Extra - non-uniform alignment
**Load the images *image1.png* and *image2.png*.**

In [27]:
import imageio

old_plane = imageio.imread('image1.png')
new_plane = imageio.imread('image2.png')
plt.figure()
plt.subplot(131)
plt.imshow(old_plane)
plt.subplot(132)
plt.imshow(new_plane)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Image2 is obtained from image1 by a non-uniform translation. Split the image into subimages, say 20 by 20 pixels, and align the subimages from image2 to the subimages of image1. Plot the shift of each subimage as a vector field, where the vectors are arrows that emmerge from the center of the subimage (if an image is 100 x 100 pixels there will be 25 subimages and 25 vectors at (10,10), (10,30), etc ...). Save the image of the vector field and submit it.
How do you think image2 was obtained from image1?**